In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time
from datetime import datetime

def crawl_faq():
    print(f"\n[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] 크롤링 시작...")
    driver = webdriver.Chrome()
    driver.get("https://www.129.go.kr/faq/faq05.do")
    wait = WebDriverWait(driver, 10)
    
    results = []
    categories = range(5, 11)
    
    try:
        for category in categories:
            # 상담분류 선택
            category_xpath = f'//*[@id="searchConsultingKeyword"]/option[{category}]'
            category_element = wait.until(EC.element_to_be_clickable((By.XPATH, category_xpath)))
            category_name = category_element.text
            category_element.click()
            print(f"\n[{datetime.now().strftime('%H:%M:%S')}] 카테고리 '{category_name}' 처리 시작")
            time.sleep(1)
            
            page_num = 1
            while True:
                print(f"\n[{datetime.now().strftime('%H:%M:%S')}] {category_name} - {page_num}페이지 처리 중...")
                
                # FAQ 테이블의 모든 행을 가져옴
                faq_rows = driver.find_elements(By.XPATH, '//*[@id="HA"]/div[3]/table/tbody/tr')
                print(f"- 현재 페이지에서 {len(faq_rows)}개의 FAQ 항목 발견")
                
                for idx, row in enumerate(faq_rows, 1):
                    try:
                        if idx % 2 == 1:  # 홀수 행
                            faq_id = row.get_attribute("id").replace("faq", "")
                            print(f"FAQ ID: {faq_id}")
                            date = driver.find_element(By.XPATH, f'//*[@id="faq{faq_id}"]/td[3]').text
                            click_xpath = f'//*[@id="faq{faq_id}"]/td[2]/p'
                            click_element = driver.find_element(By.XPATH, click_xpath)
                            driver.execute_script("arguments[0].click();", click_element)
                            time.sleep(0.5)

                            # 질문과 답변 추출
                            question = driver.find_element(
                                By.XPATH, 
                                f'//*[@id="faqList{faq_id}"]/div[1]/p'
                            ).text
                            print(f'question 수집 완료 : {question}')

                            # 답변의 맑은 고딕 폰트 span 태그들 찾기
                            answer = driver.find_element(
                                By.XPATH,
                                f'//*[@id="faqList{faq_id}"]/div[3]/div/div[1]'
                            ).text
                            print(f'answer 수집 완료 : {answer}')
                            
                            if question and answer:    
                                results.append({
                                    'question': question,
                                    'answer': answer,
                                    'metadata': {
                                        'date': date,
                                        'category': category_name,
                                        'source': 'www.129.go.kr'
                                    },
                                })
                            else:
                                assert False, "질문 또는 답변이 누락됨"
                            

                    except Exception as e:
                        print(f"FAQ 처리 실패: {str(e)}")
                        continue
                
                # 다음 페이지 확인
                
                try:
                    next_page = driver.find_element(By.XPATH, f'//*[@id="paging"]/ul/li[{3+page_num}]/a')
                    if not next_page.is_enabled():
                        print(f"[{datetime.now().strftime('%H:%M:%S')}] {category_name} - 마지막 페이지 도달")
                        break
                    next_page.click()
                    page_num += 1
                    time.sleep(2)
                except:
                    print(f"[{datetime.now().strftime('%H:%M:%S')}] {category_name} - 다음 페이지 없음")
                    break
            
            print(f"[{datetime.now().strftime('%H:%M:%S')}] 카테고리 '{category_name}' 처리 완료")
            print(f"현재까지 총 {len(results)}개의 FAQ 수집됨")
    
    except Exception as e:
        print(f"\n[{datetime.now().strftime('%H:%M:%S')}] 크롤링 중 오류 발생: {str(e)}")
    
    finally:
        driver.quit()
        file_path = f'faq_results_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
        print(f'결과 갯수 : {len(results)}')
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)

if __name__ == "__main__":
    results_df = crawl_faq()



[2024-12-11 08:47:05] 크롤링 시작...

[08:47:12] 카테고리 '장애인등록' 처리 시작

[08:47:13] 장애인등록 - 1페이지 처리 중...
- 현재 페이지에서 20개의 FAQ 항목 발견
FAQ ID: 14801
question 수집 완료 : 온라인으로 장애인 등록 신청 가능한가요?
answer 수집 완료 : '복지로(bokjiro.go.kr)에 로그인* 하여 장애인등록 신청이 가능합니다.
* 신청인이 미성년인 경우 신청인과 주소가 같은 부모가 로그인하여 대리 신청 가능

다만, 온라인 신청일부터 30일 내에 해당 장애유형에 대한 필수 첨부서류를 우편 또는 방문을 통해 관할 읍면동 주민센터에 제출해야 하며, 필수 첨부서류가 관할 읍면동 주민센터에 제출된 날이 장애인 등록 신청일이 됩니다. 만일 30일 내에 필수 첨부서류가 제출되지 않으면 온라인 신청은 자동 취소됩니다.

더 자세한 사항은 보건복지상담센터(국번없이 ☎129)로 연락주시기 바랍니다.
FAQ ID: 14800
question 수집 완료 : 장애인등록증 진위확인 방법이 있나요?
answer 수집 완료 : 장애인이 제시하는 장애인등록증의 진위확인이 필요한 경우, 누구나 '복지로(bokjiro.go.kr)'에 로그인하고, 확인하고자 하는 장애인등록증 정보(성명, 주민등록번호(외국인등록번호), 발급일자, 조회기관(선택)) 입력한 후 "진위확인" 버튼 클릭하면 진위확인 메시지*가 표출됩니다.
* ‘입력하신 내용은 장애인등록 정보와 일치합니다.’ 또는 ‘입력하신 내용은 장애인등록 정보와 일치하지 않습니다. 자세한 사항은 가까운 읍면동 주민센터에 문의하시기 바랍니다.’

더 자세한 사항은 보건복지상담센터(국번없이 ☎129)로 연락주시기 바랍니다.
   
FAQ ID: 1403
question 수집 완료 : 장애진단서 발급비용은 어떠한 경우에 지원받을 수 있나요?
answer 수집 완료 : 기초생활수급자가 신규로 장애인 등록을 신청하거나 재판정*으로 재진단을 받는 등록 장애인 중 

In [4]:
url = 'https://www.129.go.kr/faq/faq05.do?pageIndex=1&searchUseYn=N&searchSort=1&searchConsultingKeyword=D05&searchCondition=0&searchKeyword='
driver = webdriver.Chrome()

driver.get(url)
print(driver.find_element(By.XPATH, f'//*[@id="faq14801"]/td[3]').text)
# faq_rows = driver.find_elements(By.XPATH, '//*[@id="HA"]/div[3]/table/tbody/tr')
# print(len(faq_rows))

2024-11-04


In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def test_faq_crawling():
    print("테스트 시작...")
    driver = webdriver.Chrome()
    
    try:
        # URL 접속
        url = "https://www.129.go.kr/faq/faq05.do?pageIndex=1&searchUseYn=N&searchSort=1&searchConsultingKeyword=D05&searchCondition=0&searchKeyword="
        driver.get(url)
        print("페이지 로딩 완료")
        
        # 잠시 대기
        time.sleep(2)
        
        # FAQ 제목 클릭
        click_xpath = '//*[@id="faq14801"]/td[2]/p'
        click_element = driver.find_element(By.XPATH, click_xpath)
        print("클릭할 요소 찾음")
        
        # JavaScript로 클릭 실행
        driver.execute_script("arguments[0].click();", click_element)
        print("클릭 실행됨")
        
        # 내용이 로드될 때까지 잠시 대기
        time.sleep(1)
        
        # 질문 내용 가져오기
        content_xpath = '//*[@id="faqList14801"]/div[1]/p'
        element = driver.find_element(By.XPATH, content_xpath)
        
        # 질문 텍스트 출력
        print("\n=== 질문 내용 ===")
        print(element.text)
        print("==================")
        
        # 답변 가져오기 (XPath 수정)
        answer_xpath = '//*[@id="faqList14801"]/div[3]/div/div[1]'
        try:
            answer_element = driver.find_element(By.XPATH, answer_xpath)
            print("\n=== 답변 내용 (전체) ===")
            print(answer_element.text)
            print("==================")
            
        except Exception as e:
            print(f"답변 추출 중 오류: {str(e)}")

        try:
            driver.find_element(By.CSS_SELECTOR, "#paging > ul > li.next > a").click()
            print("다음 페이지로 이동")
            time.sleep(2)
        except:
            print("다음 페이지 없음")
        
    except Exception as e:
        print(f"오류 발생: {str(e)}")
    
    finally:
        driver.quit()
        print("\n테스트 완료")

if __name__ == "__main__":
    test_faq_crawling()

테스트 시작...
페이지 로딩 완료
클릭할 요소 찾음
클릭 실행됨

=== 질문 내용 ===
온라인으로 장애인 등록 신청 가능한가요?

=== 답변 내용 (전체) ===
'복지로(bokjiro.go.kr)에 로그인* 하여 장애인등록 신청이 가능합니다.
* 신청인이 미성년인 경우 신청인과 주소가 같은 부모가 로그인하여 대리 신청 가능

다만, 온라인 신청일부터 30일 내에 해당 장애유형에 대한 필수 첨부서류를 우편 또는 방문을 통해 관할 읍면동 주민센터에 제출해야 하며, 필수 첨부서류가 관할 읍면동 주민센터에 제출된 날이 장애인 등록 신청일이 됩니다. 만일 30일 내에 필수 첨부서류가 제출되지 않으면 온라인 신청은 자동 취소됩니다.

더 자세한 사항은 보건복지상담센터(국번없이 ☎129)로 연락주시기 바랍니다.
다음 페이지 없음

테스트 완료


In [ ]:
!pip install requests beautifulsoup4

  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
  Using cached charset_normalizer-3.4.0-cp310-cp310-macosx_11_0_arm64.whl (120 kB)
  Using cached soupsieve-2.6-py3-none-any.whl (36 kB)

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
